<a href="https://colab.research.google.com/github/publictrain/kubota/blob/main/kubotasoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import re
import requests
import time
import random
import csv
import urllib
import os
import pprint
import time
import urllib.error
# Colabにも使ってないライブラリわかるようになる機能つけてくれ


headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"}

def main():
  ans_list = []
  ans_list_box = []
  pdf_list = []
  k=int(input("(ここに入力した数*30くらいの安保理データが手に入るよ！"))

    load_url = "https://www.securitycouncilreport.org/un-documents-all-pdfs-last-added/page/{0}".format(i)# これヘッダーがないと動かない。スクレイピングよけだろう...
    html = requests.get(load_url,headers=headers)
    soup = BeautifulSoup(html.content, "html.parser")
    elems = soup.select("h7 > a")# elemsの型はBs特有のもの、h7 > aでネストの探索

    for elem in elems:
      if "pdf" in str(elem):
        pdf_list.append(str(elem.get("href")))#　elemの型はBs特有のもの、それをキャストしてる
    wait_xs()
  print(pdf_list)

  for j in range(len(pdf_list)):
    req = urllib.request.Request(pdf_list[j], headers=headers)# 403よけHeader
    download_file(req, "/content/drive/MyDrive/Colab Notebooks/kubota/gomi/pekopeko{0}.pdf".format(j))

def wait_xs():
  sec = random.uniform(0.1, 0.5)
  time.sleep(sec)

def download_file(req, dst_path):
    try:
        with urllib.request.urlopen(req) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)


if __name__ == "__main__":
    main()




(ここに入力した数の安保理データが手に入るよ！3
['https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_PV_9035.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_2022_391.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_2022_382.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_2022_381.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/s_2022_276.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_PV.9021.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_2022_345.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/N2232578.pdfN2232578.pdf', 'https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S:2022:338.pdf', 'https://www